In [1]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess

model = AutoModel(
    model="/mnt/e/LLM/ASR/paraformer",
    vad_model="/mnt/e/LLM/ASR/fsmn_vad",
    punc_model="/mnt/e/LLM/ASR/punc_ct",
    spk_model="/mnt/e/LLM/ASR/cam++",
    device="cuda:0",
    )

funasr version: 1.2.7.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel


You are using the latest version of funasr-1.2.7


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.385 seconds.
DEBUG:jieba:Loading model cost 0.385 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
import os
audio_dir = '/mnt/e/LLM/audio_data/'
audio_files = os.listdir(audio_dir)
print('number of audio files:', len(audio_files))

for audio_name in audio_files:
    # 判断output文件夹是否有同名的txt文件，如果有，跳过
    output_name = audio_name.split('.mp3')[0] + '.txt'
    output_path = os.path.join('/mnt/e/LLM/output/', output_name)
    if os.path.exists(output_path):
        print(output_path, 'exists, skip')
        continue
    # 加载模型
    audio_path = os.path.join(audio_dir, audio_name)
    res = model.generate(input=audio_dir+audio_name, 
            batch_size_s=600,
            language='zn',
            device='cuda:0',
            beam_size=4,
            disable_pbar=True,
            )
    # 保存结果
    with open(output_path, 'w') as f:
        text = rich_transcription_postprocess(res[0]["text"])
        f.write(text)
    print(output_path, 'saved')

number of audio files: 1


rtf_avg: 0.037: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]                                                                                          


/mnt/e/LLM/output/南娘大赛3.txt saved


In [24]:
import pandas as pd
table_data = []
for segment in res[0]['sentence_info']:
    if isinstance(segment, dict) and 'text' in segment:
        start_time = ms_to_time_str(segment.get('start', 0))
        end_time = ms_to_time_str(segment.get('end', 0))
        time_range = f"{start_time} - {end_time}"
        speaker = f"说话人 {segment.get('spk', 'unknown')}"
        text = segment.get('text', '')
        table_data.append({
            "时间段": time_range,
            "说话人": speaker,
            "内容": text
        })
if table_data:
    df = pd.DataFrame(table_data)
    print(df.to_string(index=False))
else:
    print("No data found.")

              时间段   说话人                                  内容
0:00:00 - 0:00:03 说话人 0           几乎每一个老玩家的史蒂姆库里都有着几十上百款游戏，
0:00:04 - 0:00:07 说话人 0              PS 的光盘与 NS 的卡带也是可以堆起来。
0:00:07 - 0:00:07 说话人 0                                 然而，
0:00:07 - 0:00:10 说话人 0                   你问他们是否还愿意昏天黑地开一局，
0:00:10 - 0:00:12 说话人 0                            热血沸腾战一场？
0:00:12 - 0:00:14 说话人 0                       他们很可能会摆摆手说可以，
0:00:14 - 0:00:14 说话人 0                               但没必要。
0:00:15 - 0:00:17 说话人 0                       这种情况我们俗称电子阳痿，
0:00:17 - 0:00:19 说话人 0                     再也无法像小时候那样精力饱满，
0:00:19 - 0:00:21 说话人 0                            想玩游戏的冲动。
0:00:21 - 0:00:22 说话人 0                         可以把地球打穿在工作。
0:00:22 - 0:00:23 说话人 0                                几年后，
0:00:23 - 0:00:25 说话人 0                        很多人都会患上电子阳痿。
0:00:25 - 0:00:27 说话人 0                      到了三十岁左右是疾病高发期，
0:00:27 - 0:00:30 说话人 0                   具体表现是已经实现游戏装备自由的，
0:00:30 - 0:00:32 说话人 0                 

In [25]:
res[0].keys()
# 
# res[0]['sentence_info']
# res[0]['timestamp']

dict_keys(['key', 'text', 'timestamp', 'sentence_info'])

In [ ]:
res[0]['text']

['几乎每一个老玩家的史蒂姆库里都有着几十上百款游戏，PS的光盘与NS的卡带也是可以堆起来',
 '然而，你问他们是否还愿意昏天黑地开一局，热血沸腾战一场？他们很可能会摆摆手说可以，但没必要',
 '这种情况我们俗称电子阳痿，再也无法像小时候那样精力饱满，想玩游戏的冲动',
 '可以把地球打穿在工作',
 '几年后，很多人都会患上电子阳痿',
 '到了三十岁左右是疾病高发期，具体表现是已经实现游戏装备自由的，他们需要一定心理建设，才会走到电脑面前',
 '而坐上电竞椅那一刻起就如坐针毡',
 '不管什么游戏，可能只是刚玩个开场，就不耐烦的拿起手机刷抖音、看短视频，把面前的游戏完全抛诸脑后了',
 '患此症后，部分人也许还会继续玩游戏',
 '但游戏时长没有以往那么长短，时间就进入厌倦状态，而且在游戏过程中，即使获得胜利，也得不到乐趣和满足，反而会有一种时间被荒废的负罪感',
 '我家里的游戏设备会也是非常奇了，每次更新换代也会抢先入手，但每隔一段时间都要考虑要不要丢到二手市场',
 '最终不得不承认，一个惊恐的现实',
 '在游戏面前，我可耻了，阳痿了',
 '遥想当年，在封杀游戏的浩荡浪潮中，一些学校的校长带头进行誓师大会，老师们还会钻进网吧，逮捕学生，用喊打喊杀的姿态，对所谓的电子海洛因极尽倾剿之冷事',
 '时光飞逝，在毕业后，在工作后，在结婚后，游戏上瘾的没有了多少电子阳痿的倒是不计其数，甚至很难找到一款特效游戏来拯救他们',
 '大家好，我是丰兄',
 '这期给大家聊聊电子扬痿',
 '说起对网络的爽感，最令人心潮澎湃的时刻是来自于世纪之交',
 '一九九五年五月十七日，第二十七个世界电信日邮电部正式宣布向国内社会开放计算机互联网揭露服务',
 '此后，无论是单位还是个人，只要交钱就可以登录互联网',
 '几年之后，到了九十年代末期，揭露互联网的人数暴增，网络进入流行姿态',
 '光是一个OICQ,就足以让所有人沉迷，全程都在讨论各大论坛的聊天室可以说是火到天际，看板块的名字就觉得脸红心跳',
 '无数玩家没日没夜在聊天室对话和社交学生们不想上课，上班的不想工作，麻将室生意突然清淡了，就连隔壁按摩店的技师们也变得敷衍了',
 '在当时的互联网娱乐里，能与聊天室相抗衡的就只有游戏了',
 '那时流行的是暗黑破坏神医帝国时代、星际争霸、流星与征服死亡之屋、炎龙